In [ ]:
# -------- Block 0: Mount Google Drive and create folder structure --------
from google.colab import drive
drive.mount('/content/drive')

import os

BASE_DIR = "/content/drive/MyDrive/genai_kids_room_project/synthetic_data"
CLASSES = ["sharp_object", "choking_hazard", "electrical_hazard", "chemical_danger"]


for cls in CLASSES:
    os.makedirs(os.path.join(BASE_DIR, cls, "images"), exist_ok=True)
    os.makedirs(os.path.join(BASE_DIR, cls, "labels"), exist_ok=True)

print("Folders created for all classes")


In [ ]:
# -------- Block 2: Install dependencies --------
!pip install -q diffusers transformers accelerate safetensors torch torchvision

In [ ]:
# -------- Block 3: Imports --------
import os
import torch
from diffusers import StableDiffusionXLPipeline
from tqdm import tqdm

In [ ]:
# -------- Block 5: Load SDXL model (T4 safe) --------
model_id = "stabilityai/stable-diffusion-xl-base-1.0"

pipe = StableDiffusionXLPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
)

pipe = pipe.to("cuda")
pipe.enable_attention_slicing()

In [ ]:
# -------- Block 3: Define prompts for each subclass --------

prompts = {
    "sharp_object": [
        "A single kitchen knife on plain white background, isolated object, studio lighting",
        "A pair of scissors on plain white background, isolated object, studio lighting"
    ],
    "choking_hazard": [
        "A single coin on white background, isolated",
        "A small battery on white background, isolated",
        "A small colorful ball on white background, isolated",
        "A Lego plastic piece on white background, isolated"
    ],
    "electrical_hazard": [
        "A power cable lying on white background, isolated",
        "A damaged electrical adapter on white background, isolated"
    ],
    "chemical_danger": [
        "A plastic bottle with cleaning chemical, isolated on white background",
        "A detergent bottle on white background, isolated"
    ]
}

In [ ]:
import os

# -------- Block 4: Generate images for each class and subclass --------

# Define the image generation function
def generate_images(prompt, num_images, save_folder):
    for i in tqdm(range(num_images), desc=f"Generating for '{prompt[:50]}...'"):
        image = pipe(prompt=prompt).images[0]
        image_name = f"{prompt.replace(' ', '_').replace('/', '_').replace('.', '').replace(',', '')}_{i:04d}.png"
        image_path = os.path.join(save_folder, image_name)
        image.save(image_path)

# Specify how many images per prompt
NUM_IMAGES_SUBCLASS = {
    "sharp_object": 50,        # 50 ножей + 50 ножниц
    "choking_hazard": 25,      # 25 каждого подкласса
    "electrical_hazard": 50,
    "chemical_danger": 50
}

for cls, cls_prompts in prompts.items():
    save_folder = os.path.join(BASE_DIR, cls, "images")
    os.makedirs(save_folder, exist_ok=True) # Ensure save_folder exists
    for prompt in cls_prompts:
        num = NUM_IMAGES_SUBCLASS[cls]
        generate_images(prompt, num, save_folder)
        print(f"{num} images generated for class {cls} prompt: {prompt}")

In [ ]:
# -------- Block: Auto-label synthetic images using YOLO --------
!pip install ultralytics --quiet

from ultralytics import YOLO
import os, glob
from pathlib import Path

# 1 Load pretrained YOLO model (your current best or pretrained on similar objects)
MODEL_PATH = "/content/drive/MyDrive/genai_kids_room_project/yolo_training/kids_room_hazard/weights/best.pt"
yolo_model = YOLO(MODEL_PATH)

# 2 Path to synthetic images
BASE_DIR = "/content/drive/MyDrive/genai_kids_room_project/synthetic_data"
CLASSES = ["sharp_object", "choking_hazard", "electrical_hazard", "chemical_danger"]

# 3 Auto-label each image
for cls in CLASSES:
    images_folder = os.path.join(BASE_DIR, cls, "images")
    labels_folder = os.path.join(BASE_DIR, cls, "labels")
    os.makedirs(labels_folder, exist_ok=True)

    image_files = glob.glob(os.path.join(images_folder, "*.png"))

    for img_path in image_files:
        # Run prediction
        results = yolo_model.predict(img_path, imgsz=640, conf=0.25, save=False)

        # Save bounding boxes in YOLO txt format
        txt_file = os.path.join(labels_folder, Path(img_path).stem + ".txt")
        with open(txt_file, "w") as f:
            for box in results[0].boxes:
                # Convert to YOLO format: class_id x_center y_center width height (normalized 0-1)
                cls_id = 0
                x_center = (box.xyxy[0][0] + box.xyxy[0][2]) / 2 / results[0].orig_shape[1]
                y_center = (box.xyxy[0][1] + box.xyxy[0][3]) / 2 / results[0].orig_shape[0]
                width = (box.xyxy[0][2] - box.xyxy[0][0]) / results[0].orig_shape[1]
                height = (box.xyxy[0][3] - box.xyxy[0][1]) / results[0].orig_shape[0]
                f.write(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

    print(f"Auto-labeling done for {cls}, total images: {len(image_files)}")


In [ ]:
# 4 Quick check: ensure all images have labels
for cls in CLASSES:
    images_folder = os.path.join(BASE_DIR, cls, "images")
    labels_folder = os.path.join(BASE_DIR, cls, "labels")
    images = glob.glob(os.path.join(images_folder, "*.png"))
    for img in images:
        label_file = os.path.join(labels_folder, Path(img).stem + ".txt")
        if not os.path.exists(label_file):
            print(f"Warning: No bbox for image {img}")

In [ ]:
# 5 Prepare data.yaml for YOLOv8 training
yaml_path = os.path.join(BASE_DIR, "data.yaml")
with open(yaml_path, "w") as f:
    f.write(f"train: {BASE_DIR}\n")
    f.write(f"val: {BASE_DIR}\n")
    f.write("nc: 4\n")
    f.write("names: ['sharp_object','choking_hazard','electrical_hazard','chemical_danger']\n")

print("data.yaml created at:", yaml_path)